# Select data for survey

In [1]:
!pip install transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelWithLMHead,GPT2Model,TrainingArguments, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from google.colab import drive
import pandas as pd
import random
import numpy as np
import torch
import math
import re
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Mounted at /content/drive


#Select sequences for **survey**

##INFLUENCE OF SENTIMENT-CARRYING KEYWORDS AND SENTIMENT TOKEN

In [5]:
neg=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k_sentiment_negative_test.tsv', sep="\t")
pos=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k_sentiment_positive_test.tsv', sep="\t")
combined=neg.append(pos)
combined["num_keywords_used"]=combined["num_keywords"]-combined["num_keywords_not_used"]
combined["succses_rate"]=combined["num_keywords_used"]/combined["num_keywords"]


<ipython-input-5-c8c078b0a304>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined=neg.append(pos)


In [ ]:
positive=combined[combined.sentiment=="positive"]
pos_neg=positive[positive.keywords_group=="negative"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
pos_neg["text"].head(1).values,pos_neg["keywords"].head(1).values

(array(['[positive] Wir waren in diesem Hotel. Wir werden das Hotel wieder buchen. Das Essen ist sehr gut und ausreichend. Die Atmosphäre ist toll und das Lobby schrecklich verschließend. Das Personal ist sehr freundlich und hilfsbereit. Man merkt immer wieder das die Gäste sich wohl fühlen. Das Personal ist auch sehr nett und hat uns jeden Tag mit einer netten Art und einem leckeren Essen versorgt. Die Bar im Hotel ist sehr groß und gut. Wir haben uns dort sehr wohl gefühlt und wir würden es immer wieder buchen. Die Zimmer sind einfach nur schön und sauber.'],
       dtype=object),
 array(["['schrecklich', 'verschließen', 'Lobby']"], dtype=object))

In [ ]:
positive=combined[combined.sentiment=="positive"]
pos_pos=positive[positive.keywords_group=="positive"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
pos_pos["text"].head(1).values,pos_pos["keywords"].head(1).values,pos_pos["num_keywords_used"].head(1).values

(array(['[positive] Wir waren in diesem Hotel. Das Personal war sehr zuvorkommend und freundlich. Auch die Zimmer wurden jeden Tag gründlich gereinigt.Die Bettenkissen waren sehr angenehm und gut Kissen strahlend sauber. Alle vertrauenswürdig und freundlich.Wir hatten Zimmer mit Meerblick. Wir hatten auch ein sehr großes Zimmer. Die Sauberkeit im Bad war sehr gut. Man hatte immer Platz für seine Sachen. Auch das Hotel ist sehr sauber gewesen. Die Zimmer waren sehr sauber, das Personal ist sehr freundlich und hilfsbereit und das Animationsprogramm war sehr gut, das Essen war auch gut. Der'],
       dtype=object),
 array(["['vertrauenswürdig', 'strahlend', 'Kissen']"], dtype=object),
 array([3]))

In [ ]:
positive=combined[combined.sentiment=="positive"]
pos_mix=positive[positive.keywords_group=="mixed"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
pos_mix["text"].head(1).values,pos_mix["keywords"].head(1).values,pos_mix["num_keywords_used"].head(1).values

(array(['[positive] Wir waren in diesem Hotel. Das Hotel ist sehr schön und man fühlt sich sehr wohl. Die Anlage ist sehr gepflegt und sauber. Die Zimmer sind sehr schön und mit allem ausgestattet was man braucht. Wir haben auch einen Eindruck von schrecklich vertrauenswürdigem Essen und Getränken bekommen. Das Hotelpersonal ist nett und man muss sich keine Sorgen machen, man kann alles mit einem Drink trinken. Das Hotel ist auch schön ruhig. Es gibt einen Strand. Und wenn man einen hat, wird man sofort abgeholt. Das Personal ist freundlich, hilfsbereit und hilfsbereit.'],
       dtype=object),
 array(["['vertrauenswürdig', 'schrecklich', 'Eindruck']"], dtype=object),
 array([3]))

In [ ]:
nagative=combined[combined.sentiment=="negative"]
neg_neg=nagative[nagative.keywords_group=="negative"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
neg_neg["text"].head(1).values,neg_neg["keywords"].head(1).values

(array(['[negative] Wir waren in diesem Hotel. Das Personal ist nicht freundlich und total unfreundlich, auch das Kissen schrecklich. Wenn man sich beschweren will, dann wird einem das gar nicht erklärt. Ich habe das Hotel gewechselt. Das Hotel ist sehr alt und sehr schmutzig. Handtücher werden nicht gewechselt, obwohl die Gäste in der gleichen Kategorie auch noch die Handtücher aufhängen. Das Essen ist ungenießbar. Die Tische und Stühle sind mit Schimmel verschmiert. Die Getränke werden in Plastikbechern serviert. Das Essen ist auch sehr schlecht. Es gibt nur'],
       dtype=object),
 array(["['schrecklich', 'unfreundlich', 'Kissen']"], dtype=object))

In [ ]:
nagative=combined[combined.sentiment=="negative"]
neg_pos=nagative[nagative.keywords_group=="positive"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
neg_pos["text"].head(1).values,neg_pos["keywords"].head(1).values

(array(['[negative] Wir waren in diesem Hotel. Das Zimmer war Bett fröhlich strahlend. Die Dusche war schmutzig. In der Badewanne war Schimmel. Das Bett war nur zusammengeschoben und man konnte nur durch die Zimmergänge laufen. Der Boden im Bad war dreckig. Schimmel im Bad. Die Betten waren durchgelegen und die Ränder mit schwarzen Flecken. Die Handtücher waren sehr schmutzig. Ich war froh das ich dieses Zimmer gewechselt hatte. Das Bett war nicht in Ordnung. Das Bett wurde nur sehr sporadisch gewechselt. Es stank stark im Raum und es war'],
       dtype=object),
 array(["['strahlend', 'fröhlich', 'Bett']"], dtype=object))

In [ ]:
nagative=combined[combined.sentiment=="negative"]
neg_mix=nagative[nagative.keywords_group=="mixed"].sort_values(by=["succses_rate","perplexity"], ascending=[False, True])
neg_mix["text"].head(1).values,neg_mix["keywords"].head(1).values

(array(['[negative] Wir waren in diesem Hotel. Die Zimmer waren grausam alt und alt, sie waren nicht einmal sauber. Das Bett war alt und durchgelegen und das Kind fröhlich auf dem Spielplatz. Das Bett war nur mit einem dünnen Tuch oder Lappen zu waschen. Auch das Bad war klein und die Fugen in den Fliesen total durchgelegen. Schimmel auf den Armaturen und auf den Fliesen. Das Bad war sehr dreckig und die Armaturen nicht sauber. Wenn man die Toilettenspülung aus dem Wasserhahn herausdrehen wollte, da kam man sich mit dem Kopf in'],
       dtype=object),
 array(["['fröhlich', 'grausam', 'Kind']"], dtype=object))

## Sentiment analysis (C and D)

In [6]:
tokenizer_for_perplex = AutoTokenizer.from_pretrained("distilbert-base-german-cased")
model_for_perplex = AutoModelWithLMHead.from_pretrained("distilbert-base-german-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/479k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/270M [00:00<?, ?B/s]

In [16]:
def generate_sentiment1(sentiment, model_name,tokenizer_for_perplex,model_for_perplex):
  df=pd.DataFrame(columns=["sentiment","text","perplexity"])
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelWithLMHead.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  generation_kwargs = {
    "min_length": -1,
    "top_k": 15,
    "top_p":0,# 0.5,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 100,
   # "eos_token_id": -1,
    "temperature":5.0
  #  "num_beams":3
}
  prompt="["+sentiment+"]"+" Wir waren in diesem Hotel."
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  for i in range(3):
    output = model.generate(input_ids, **generation_kwargs)
    generated_text = tokenizer.decode(output[0])
    generated_text=re.sub('\[.*?\]', '', generated_text)
    tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
    tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
    loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
    df.loc[i,"sentiment"]=sentiment
    df.loc[i,"text"]=generated_text
    df.loc[i,"perplexity"]=math.exp(loss)
    print(generated_text)

  return df.sort_values(by=["perplexity"],ascending=True)#.head(1)["text"].values[0]

In [17]:
sft1_rl_1="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs"
print("positive")
sft1_rl_1_pos=generate_sentiment1("positive", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_pos.head(1)["text"].values[0])
print("negative")
sft1_rl_1_neg=generate_sentiment1("negative", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_neg.head(1)["text"].values[0])
sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)
#sft1_rl_1.to_csv('/content/drive/MyDrive/Masterthesis/Results/sentiment_survey_sft1_rl_1.tsv', sep="\t")

positive


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Das Personal war sehr freundlich und hilfsbereit. Die Zimmer wurden jeden Tag gereinigt und das Zimmer war immer sehr sauber. Das Essen war sehr gut und abwechslungsreich. Es gab jeden Tag das gleiche und es gab immer eine Auswahl. Das Personal war sehr freundlich und hilfsbereit. Das Zimmer war immer sehr sauber und das Personal war immer sehr hilfsbereit. Das Essen war sehr gut und abwechslungsreich. Das Personal war immer sehr freundlich und hilfsbereit. Das Zimmer war immer sehr sauber und das Personal war immer sehr hilfsbereit. Das Zimmer war immer
 Wir waren in diesem Hotel. Das Personal war sehr freundlich und hilfsbereit. Die Zimmer wurden jeden Tag gereinigt und das Zimmer war immer sehr sauber. Das Essen war sehr gut und abwechslungsreich. Es gab jeden Tag das gleiche und es gab immer eine Auswahl. Das Personal war sehr freundlich und hilfsbereit. Das Zimmer war immer sehr sauber und das Personal war immer sehr hilfsbereit. Das Essen war sehr gut 

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Das Essen war ungenießbar. Es gab nur eine Sorte Wurst und eine Sorte Käse. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken
 Wir waren in diesem Hotel. Das Essen war ungenießbar. Es gab nur eine Sorte Wurst und eine Sorte Käse. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken und nicht genießbar. Das Fleisch war sehr trocken und nicht genießbar. Das Gemüse war sehr trocken
 Wir waren in diesem Hotel. Das Essen war 

<ipython-input-17-04d1ae14a852>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)


In [42]:
sft1_rl_1.perplexity.mean()

1.063390464080164

In [ ]:
sft1_rl_1.perplexity.mean()

In [73]:
def generate_sentiment(sentiment, model_name,tokenizer_for_perplex,model_for_perplex):
  df=pd.DataFrame(columns=["sentiment","text","perplexity"])
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelWithLMHead.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  generation_kwargs = {
    "min_length": -1,
    "top_k": 0,
    "top_p": 0.5,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 100,
    "eos_token_id": -1,
  #  "num_beams":3
}
  prompt="["+sentiment+"]"+" Wir waren in diesem Hotel."
  input_ids = tokenizer.encode(prompt, return_tensors="pt")
  for i in range(3):
    output = model.generate(input_ids, **generation_kwargs)
    generated_text = tokenizer.decode(output[0])
    generated_text=re.sub('\[.*?\]', '', generated_text)
    tokenize_input = tokenizer_for_perplex.tokenize(generated_text)
    tensor_input = torch.tensor([tokenizer_for_perplex.convert_tokens_to_ids(tokenize_input)])
    loss, logits = model_for_perplex(tensor_input, labels=tensor_input)[:2]
    df.loc[i,"sentiment"]=sentiment
    df.loc[i,"text"]=generated_text
    df.loc[i,"perplexity"]=math.exp(loss)
    print(generated_text)
  return df.sort_values(by=["perplexity"],ascending=True)#.head(1)["text"].values[0]

In [32]:
sft1_rl_1="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs"
print("positive")
sft1_rl_1_pos=generate_sentiment("positive", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_pos.head(1)["text"].values[0])
print("negative")
sft1_rl_1_neg=generate_sentiment("negative", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_neg.head(1)["text"].values[0])
sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)
#sft1_rl_1.to_csv('/content/drive/MyDrive/Masterthesis/Results/sentiment_survey_sft1_rl_1.tsv', sep="\t")

positive


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Es war eine sehr schöne Zeit. Wir würden wiederkommen. Das Personal war sehr freundlich und hilfsbereit. Das Essen war sehr gut und abwechslungsreich. Die Zimmer waren sehr sauber und die Zimmer wurden jeden Tag gründlich gereinigt. Das Personal war sehr freundlich und hilfsbereit. Die Zimmer waren sehr geräumig und auch die Betten waren sehr beqüm. Die Reinigung der Zimmer war auch sehr gut. Die Mitarbeiter waren sehr hilfsbereit und immer freundlich. Die Lage war auch sehr schön. Wir würden wiederkommen. Wir kommen gerne wieder.
 Wir waren in diesem Hotel. Die Lage ist gut. Der Strand ist auch gut. Das Personal ist sehr freundlich und hilfsbereit. Wir hatten ein Zimmer im dritten Stock. Die Zimmer sind sehr groß und sehr sauber. Wir hatten ein sehr großes Bad. Das Zimmer war sehr sauber und wir hatten eine kleine Küchenzeile. Das Personal war sehr nett und hilfsbereit. Wir hatten Halbpension und es gab jeden Tag eine Auswahl an leckeren Speisen. Wir haben 

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Das Essen war schlecht. Es gab immer das gleiche. Es gab nur das gleiche. Das Wasser war kalt. Das Wasser im Pool war sehr kalt. Das Wasser im Pool war sehr schmutzig. Das Personal war sehr unfreundlich. Das Personal im Restaurant war sehr unfreundlich. Das Personal im Speisesaal war sehr unfreundlich. Das Essen war sehr schlecht. Es gab nur ein Gericht. Das Gemüse war sehr kalt. Das Fleisch war sehr kalt. Das Fleisch war sehr kalt. Das Fleisch war sehr kalt.
 Wir waren in diesem Hotel. Das Essen war eine Zumutung. Die Getränke waren ungenießbar. Das Personal war unfreundlich. Das Zimmer war dreckig und nicht sauber. Es gab keine Reinigung. Die Handtücher wurden nicht gewechselt. Das Bett wurde nicht einmal gewechselt. Das Zimmer war dreckig. Die Bettwäsche wurde nur einmal gewechselt. Das Bad war sehr dreckig. Die Badewanne war dreckig. Die Fliesen im Bad waren schmutzig. Das Bett war dreckig. Die Handtücher wurden nicht gewechselt. Das Essen war eine Zumut

<ipython-input-32-3af5655c9838>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)


In [33]:
sft1_rl_1#.text.values#["perplexity"].mean()

,sentiment,text,perplexity
2,positive,Wir waren in diesem Hotel. Die Zimmer sind se...,1.050525
1,positive,Wir waren in diesem Hotel. Die Lage ist gut. ...,1.055254
7,positive,Wir waren in diesem Hotel. Es war alles sehr ...,1.05629
6,positive,Wir waren in diesem Hotel. Die Zimmer sind gr...,1.062634
0,positive,Wir waren in diesem Hotel. Es war eine sehr s...,1.063854
9,positive,Wir waren in diesem Hotel. Die Lage ist wirkl...,1.06666
5,positive,Wir waren in diesem Hotel. Es war sehr schön....,1.076841
8,positive,Wir waren in diesem Hotel. Es war eine sehr s...,1.078659
3,positive,Wir waren in diesem Hotel. Das Personal war i...,1.101344
4,positive,Wir waren in diesem Hotel. Die Zimmer sind se...,1.125836


### GPT

In [ ]:
gpt="benjamin/gpt2-wechsel-german"
print("positive")
gpt_pos=generate_sentiment("positive", gpt,tokenizer_for_perplex,model_for_perplex)
print(gpt_pos.head(1)["text"].values[0])
print("negative")
gpt_neg=generate_sentiment("negative", gpt,tokenizer_for_perplex,model_for_perplex)
print(gpt_neg.head(1)["text"].values[0])
gpt=gpt_pos.append(gpt_neg)
gpt.to_csv('/content/drive/MyDrive/Masterthesis/Results/sentiment_survey_gpt.tsv', sep="\t")


positive


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


 Wir waren in diesem Hotel. Das Zimmer war sauber und sehr komfortabel. Die Lage war ideal. Das Hotel ist nur wenige Gehminuten vom Strand entfernt. Das Personal war sehr freundlich und hilfsbereit. Wir würden wieder hier übernachten.
Sehr geehrter Herr Blümel, Vielen Dank für Ihre Bewertung. Es freut uns sehr, dass Sie sich für den Aufenthalt bei uns im Haus entschieden haben. Wir hoffen, Sie bald wieder bei uns begrüßen zu dürfen. Mit freundlichen Grüßen Ihr Hotelmanager Familie Roth und Ihr Team
Sehr geehrter Herr Blümel,
negative


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


 Wir waren in diesem Hotel. Die Zimmer waren sehr schön und sauber. Das Personal war sehr freundlich und hilfsbereit. Das Frühstück war sehr gut und abwechslungsreich. Das...”
“Sehr gutes Hotel, sehr gute Lage. Alles ist fußläufig erreichbar. Sehr freundliches Personal. Sehr gutes Frühstück. Wir kommen gerne wieder. Vielen Dank für Ihre Empfehlung!
Das Hotel liegt sehr zentral, sehr ruhig. Sehr freundliches Personal. Wir haben uns sehr wohl gefühlt.
Das Hotel ist sehr gut gelegen. Der Service war sehr gut. Die Zimmer sind sehr


<ipython-input-59-952de7f1eccd>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gpt=gpt_pos.append(gpt_neg)


### Supervised Fine Tuning One Epoch

In [ ]:
sft1="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_with_tokens_big_1_epochs"
print("positive")
sft1_pos=generate_sentiment("positive", sft1,tokenizer_for_perplex,model_for_perplex)
print(sft1_pos.head(1)["text"].values[0])
print("negative")
sft1_neg=generate_sentiment("negative", sft1,tokenizer_for_perplex,model_for_perplex)
print(sft1_neg.head(1)["text"].values[0])
sft1=sft1_pos.append(sft1_neg)
sft1.to_csv('/content/drive/MyDrive/Masterthesis/Results/sentiment_survey_sft1.tsv', sep="\t")

positive


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


 Wir waren in diesem Hotel. Es ist ein Familienhotel. Der Altersdurchschnitt ist gemischt. Die Kinder haben wir nur einmal in einem Kinderclub besucht. Die Animation war gut. Es gab keine Musik. Man konnte immer etwas unternehmen. Das Essen war gut. Es gab auch immer ein Salatbuffet. Wir haben es aber nicht genutzt. Das Frühstück war gut. Es gab verschiedene Brotsorten, verschiedene Wurstsorten, verschiedene Käse, Marmelade, Honig, Müsli, Joghurt, Eier, Wurst, verschiedene Brotsorten, verschiedene Brötchen, Eier
negative


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


 Wir waren in diesem Hotel. Im ersten Moment war das Hotel schön, aber dann haben wir uns auf der Terrasse in der Mitte mit den vielen Mücken aufgehalten. Die Einrichtung war schon alt und altbacken. Wir haben es nicht genutzt. Es war sehr laut und laut. Das Zimmer war sehr klein. Das Bad war dreckig. Die Dusche war dreckig und auch die Toilette war dreckig. Es gab keine Klimaanlage. Wir haben uns beschwert und wurden sogar auf den Nachbarflur geschickt. Wir haben das Hotel gewechselt. Die Mitarbeiter an der


<ipython-input-58-63c339b32051>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sft1=sft1_pos.append(sft1_neg)


In [ ]:
sft1_rl_1="/content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs"
print("positive")
sft1_rl_1_pos=generate_sentiment("positive", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_pos.head(1)["text"].values[0])
print("negative")
sft1_rl_1_neg=generate_sentiment("negative", sft1_rl_1,tokenizer_for_perplex,model_for_perplex)
print(sft1_rl_1_neg.head(1)["text"].values[0])
sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)
sft1_rl_1.to_csv('/content/drive/MyDrive/Masterthesis/Results/sentiment_survey_sft1_rl_1.tsv', sep="\t")

positive


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Das Personal war sehr freundlich und hilfsbereit. Die Zimmer wurden jeden Tag gereinigt. Die Animation war auch sehr nett und hilfsbereit. Das Essen war auch sehr gut. Es gab immer genug Obst und Gemüse. Es gab immer ausreichend Auswahl an warmen Speisen. Auch die Nachspeisen waren immer lecker. Das Zimmer war immer sehr sauber und das Personal war sehr nett und hilfsbereit. Die Animation war auch sehr nett und hilfsbereit. Das Essen war auch immer sehr lecker und auch die Auswahl war sehr gut. Das Personal ist sehr nett
negative


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at /content/drive/MyDrive/Masterthesis/Models/german_gpt2_sft_1_epoch_rl_1epochs were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

 Wir waren in diesem Hotel. Das Essen war eine Zumutung. Es gab nur eine Sorte Fleisch, das nicht durchgegart war. Das Fleisch war so trocken, dass es aussah als wäre es schon aufgeweicht. Es gab jeden Tag das selbe. Die Atmosphäre war sehr schlecht. Es gab keinen Service. Es gab nur einen Kellner, der sich um die Gäste kümmerte. Der war auch sehr unfreundlich. Es gab kein warmes Essen. Es gab nur eine Sorte Wurst. Das Brot war eine Zumutung. Es gab kein frisches Brot


<ipython-input-66-7c3bea355a48>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sft1_rl_1=sft1_rl_1_pos.append(sft1_rl_1_neg)


##DECODING STRATEGIES (ROWS E and F)

BASED ON SFT 1 RL 1

In [ ]:
topp=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_p_11.tsv', sep="\t")
topk=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/top_k11.tsv', sep="\t")
gready=pd.read_csv('/content/drive/MyDrive/Masterthesis/Results/gready_search11.tsv', sep="\t")

In [ ]:
gready[(gready.sentiment=="[negative]") & (gready.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(10)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
19,19,[negative],[negative] Wir waren in diesem Hotel. Die Fens...,Wir waren in diesem Hotel. Die Fenster waren ...,1.064179,"['Rechnung', 'Fenster', 'Heizung']",3,0
38,38,[negative],[negative] Wir waren in diesem Hotel. Preis-Le...,Wir waren in diesem Hotel. Preis-Leistung nic...,1.070788,"['Handtuch', 'Preis', 'Hilfe']",3,0
0,0,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.073434,"['Fitness', 'Atmosphäre', 'Hilfe']",3,0
17,17,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.075685,"['Aussicht', 'Schwimmbad', 'Fitness']",3,0
4,4,[negative],[negative] Wir waren in diesem Hotel. Das Bade...,Wir waren in diesem Hotel. Das Badezimmer war...,1.076802,"['Sauberkeit', 'Badewanne', 'Eindruck']",3,0
3,3,[negative],[negative] Wir waren in diesem Hotel. Die Saub...,Wir waren in diesem Hotel. Die Sauberkeit der...,1.077485,"['Atmosphäre', 'Erreichbarkeit', 'Lage']",3,0
5,5,[negative],[negative] Wir waren in diesem Hotel. Personal...,Wir waren in diesem Hotel. Personal war sehr ...,1.084891,"['Atmosphäre', 'Personal', 'Handtuch']",3,0
37,37,[negative],[negative] Wir waren in diesem Hotel. Das Hote...,Wir waren in diesem Hotel. Das Hotel ist sehr...,1.092850,"['Bett', 'Wellness', 'Preis']",3,0
1,1,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.098879,"['Fitness', 'Empfang', 'Badezimmer']",3,0
15,15,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war eine...,1.106625,"['Badewanne', 'Atmosphäre', 'Lage']",3,0


In [ ]:
topk[(topk.sentiment=="[negative]") & (topk.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(10)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
32,32,[negative],[negative] Wir waren in diesem Hotel. Personal...,Wir waren in diesem Hotel. Personal im allgem...,1.061130,"['Handtuch', 'Personal', 'Parkplatz']",3,0
38,38,[negative],[negative] Wir waren in diesem Hotel. Die Zimm...,Wir waren in diesem Hotel. Die Zimmer sind se...,1.061402,"['Handtuch', 'Preis', 'Hilfe']",3,0
17,17,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war sehr...,1.068926,"['Aussicht', 'Schwimmbad', 'Fitness']",3,0
7,7,[negative],[negative] Wir waren in diesem Hotel. Nicht zu...,"Wir waren in diesem Hotel. Nicht zum Essen, n...",1.072053,"['Preis', 'Klimaanlage', 'Haustier']",3,0
3,3,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.072786,"['Atmosphäre', 'Erreichbarkeit', 'Lage']",3,0
13,13,[negative],[negative] Wir waren in diesem Hotel. Sauberke...,Wir waren in diesem Hotel. Sauberkeit und Lei...,1.077529,"['Hilfe', 'Leistungs', 'Sauberkeit']",3,0
1,1,[negative],[negative] Wir waren in diesem Hotel. Die Bade...,Wir waren in diesem Hotel. Die Badezimmer sin...,1.080842,"['Fitness', 'Empfang', 'Badezimmer']",3,0
0,0,[negative],[negative] Wir waren in diesem Hotel. Atmosphä...,Wir waren in diesem Hotel. Atmosphäre und Fit...,1.090217,"['Fitness', 'Atmosphäre', 'Hilfe']",3,0
5,5,[negative],[negative] Wir waren in diesem Hotel. Personal...,Wir waren in diesem Hotel. Personal in diesem...,1.093171,"['Atmosphäre', 'Personal', 'Handtuch']",3,0
4,4,[negative],[negative] Wir waren in diesem Hotel. Sauberke...,Wir waren in diesem Hotel. Sauberkeit und Zim...,1.093309,"['Sauberkeit', 'Badewanne', 'Eindruck']",3,0


In [ ]:
topp[(topp.sentiment=="[negative]") & (topp.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(10)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
1,1,[negative],[negative] Wir waren in diesem Hotel. Die Saub...,Wir waren in diesem Hotel. Die Sauberkeit war...,1.055552,"['Fitness', 'Empfang', 'Badezimmer']",3,0
3,3,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war eine...,1.067393,"['Atmosphäre', 'Erreichbarkeit', 'Lage']",3,0
8,8,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.067484,"['Kind', 'Restaurant', 'Handtuch']",3,0
47,47,[negative],[negative] Wir waren in diesem Hotel. Das Hote...,Wir waren in diesem Hotel. Das Hotel ist sehr...,1.068066,"['Aussicht', 'Restaurant', 'Wellness']",3,0
19,19,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.070843,"['Rechnung', 'Fenster', 'Heizung']",3,0
17,17,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war schl...,1.081315,"['Aussicht', 'Schwimmbad', 'Fitness']",3,0
0,0,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.083654,"['Fitness', 'Atmosphäre', 'Hilfe']",3,0
12,12,[negative],[negative] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war unge...,1.087529,"['Badewanne', 'Kaffee', 'Kind']",3,0
11,11,[negative],[negative] Wir waren in diesem Hotel. Handtuch...,Wir waren in diesem Hotel. Handtuchwechsel wa...,1.091760,"['Handtuch', 'Heizung', 'Minibar']",3,0
5,5,[negative],[negative] Wir waren in diesem Hotel. Personal...,Wir waren in diesem Hotel. Personal war nicht...,1.093531,"['Atmosphäre', 'Personal', 'Handtuch']",3,0


In [ ]:
gready[gready.sentiment=="[negative]"].head()
gready.loc[17,"text"],gready.loc[17,"keywords"]

('[negative] Wir waren in diesem Hotel. Das Essen war ungenießbar. Die Aussicht auf den Pool war sehr schlecht. Fitnessraum war auch nicht vorhanden. Schwimmbad war sehr klein und sehr schmutzig. Die Liegen waren sehr dreckig. Die Handtücher wurden nur einmal gewechselt. Das Wasser im Pool war sehr kalt. Die Handtücher wurden nur einmal gewechselt. Das Personal war sehr unfreundlich. Das Essen war sehr schlecht. Das Frühstück war ungenießbar. Das Brot war sehr trocken. Das Brot war sehr trocken. Das Brot war sehr trocken. Das Brot',
 "['Aussicht', 'Schwimmbad', 'Fitness']")

In [ ]:
topk[topk.sentiment=="[negative]"].head()
topk.loc[17,"text"],topk.loc[17,"keywords"]

('[negative] Wir waren in diesem Hotel. Das Essen war sehr schlecht. Es war sehr kalt. Das Personal war sehr unfreundlich und der Pool war auch nicht beheizt. Die Aussicht auf Schwimmbad und Fitnessraum war echt schlecht. Wir hatten einen tollen Ausblick auf den Fluss, der sich in die Höhe bewegte. Das Zimmer war klein, schmutzig und alt. Die Bettwäsche wurde jeden Tag gewechselt. Wenn das Wetter zu heiß wurde, musste man die Kleidung ausziehen, da diese nass war. Das Bad war dreckig. Das Wasser wurde auch nicht gewechselt.',
 "['Aussicht', 'Schwimmbad', 'Fitness']")

In [ ]:
topp[topp.sentiment=="[negative]"].head()
topp.loc[17,"text"], topp.loc[17,"keywords"]

('[negative] Wir waren in diesem Hotel. Das Essen war schlecht. Es gab immer das gleiche. Das Personal war sehr unfreundlich. Die Zimmerreinigung war auch nicht gut. Die Aussicht auf das Meer war sehr schlecht. Das Schwimmbad war sehr klein und sehr schmutzig. Fitnessraum war auch nicht vorhanden. Das Hotel ist sehr veraltet. Die Sauberkeit der Zimmer ist auch nicht sehr gut. Das Bad ist sehr alt und schimmlig. Die Fenster sind nur für Erwachsene und Kinder geeignet. Die Handtücher werden nicht gewechselt. Die Betten sind sehr durchgelegen.',
 "['Aussicht', 'Schwimmbad', 'Fitness']")

In [ ]:
gready[(gready.sentiment=="[positive]") & (gready.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(3)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
30,30,[positive],[positive] Wir waren in diesem Hotel. Die Zimm...,Wir waren in diesem Hotel. Die Zimmer sind se...,1.048728,"['Wellness', 'Lage', 'Sauberkeit']",3,0
16,16,[positive],[positive] Wir waren in diesem Hotel. Das Hote...,Wir waren in diesem Hotel. Das Hotel ist sehr...,1.062819,"['Wellness', 'Kind', 'Sauberkeit']",3,0
41,41,[positive],[positive] Wir waren in diesem Hotel. Eindruck...,Wir waren in diesem Hotel. Eindruckvoll und s...,1.065127,"['Eindruck', 'Badewanne', 'Kaffee']",3,0


In [ ]:
 topk[(topk.sentiment=="[positive]") & (topk.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(3)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
30,30,[positive],[positive] Wir waren in diesem Hotel. Die Lage...,Wir waren in diesem Hotel. Die Lage war super...,1.062778,"['Wellness', 'Lage', 'Sauberkeit']",3,0
16,16,[positive],[positive] Wir waren in diesem Hotel. Das Pers...,Wir waren in diesem Hotel. Das Personal ist s...,1.063897,"['Wellness', 'Kind', 'Sauberkeit']",3,0
10,10,[positive],[positive] Wir waren in diesem Hotel. Das Stud...,Wir waren in diesem Hotel. Das Studio war sau...,1.064815,"['Atmosphäre', 'Studio', 'Empfang']",3,0


In [ ]:
topp[(topp.sentiment=="[positive]") & (topp.num_keywords_not_used==0)].sort_values(by=["perplexity"],ascending=True).head(3)

,Unnamed: 0,sentiment,text,gen_text,perplexity,keywords,num_keywords,num_keywords_not_used
30,30,[positive],[positive] Wir waren in diesem Hotel. Das Hote...,Wir waren in diesem Hotel. Das Hotel ist sehr...,1.046483,"['Wellness', 'Lage', 'Sauberkeit']",3,0
25,25,[positive],[positive] Wir waren in diesem Hotel. Das Pers...,Wir waren in diesem Hotel. Das Personal war s...,1.047696,"['Hilfe', 'Dusche', 'Empfang']",3,0
41,41,[positive],[positive] Wir waren in diesem Hotel. Das Esse...,Wir waren in diesem Hotel. Das Essen war sehr...,1.056896,"['Eindruck', 'Badewanne', 'Kaffee']",3,0


In [ ]:
gready[gready.sentiment=="[positive]"].head()
gready.loc[30,"text"], gready.loc[30,"keywords"]

('[positive] Wir waren in diesem Hotel. Die Zimmer sind sehr groß und sauber. Die Lage ist super. Sauberkeit und Wellness sind sehr gut. Das Personal ist sehr freundlich und hilfsbereit. Das Essen ist sehr gut und abwechslungsreich. Das Personal ist sehr hilfsbereit und immer freundlich. Das Zimmer ist sehr groß und sauber. Das Personal ist sehr freundlich und hilfsbereit. Das Zimmer ist sehr groß und sauber. Das Personal ist sehr hilfsbereit und hilfsbereit. Das Zimmer ist sehr groß und sauber. Das Zimmer ist sehr groß und sauber. Das Personal ist sehr freundlich und hilfsbereit',
 "['Wellness', 'Lage', 'Sauberkeit']")

In [ ]:
topk[topk.sentiment=="[positive]"].head()
topk.loc[30,"text"], topk.loc[30,"keywords"]

('[positive] Wir waren in diesem Hotel. Die Lage war super, Sauberkeit gut und Wellness pur. Es war ein perfekter Urlaub, die Zimmer waren in allen Bereichen sehr sauber. Wir werden gerne wiederkommen!Das Hotel ist sehr gepflegt und der Strand war auch immer schön gepflegt. Der Weg zum Strand war auch sehr schön, der Strand war sehr sauber und es gab viele Liegen mit Sonnenschirm. Das Zimmer war in Ordnung und die Handtücher wurden gewechselt. Ein großes Dankeschön an die Rezeption, ich habe immer an die Hand genommen. Das Essen war frisch und',
 "['Wellness', 'Lage', 'Sauberkeit']")

In [ ]:
topp[topp.sentiment=="[positive]"].head()
topp.loc[30,"text"], topp.loc[30,"keywords"]

('[positive] Wir waren in diesem Hotel. Das Hotel ist sehr groß und gut ausgestattet. Die Zimmer sind sehr sauber und das Personal ist sehr freundlich und hilfsbereit. Das Essen ist sehr gut und abwechslungsreich. Die Lage ist sehr schön und das Hotel ist in der Nähe von Sauberkeit und Wellness. Wir haben uns sehr wohl gefühlt und werden in Zukunft wieder kommen. Wir würden das Hotel auf jeden Fall weiter empfehlen. Das Personal ist sehr freundlich und hilfsbereit. Wir haben uns sehr wohl gefühlt und würden jederzeit wiederkommen. Es gibt eine kleine Auswahl an Getränken. Das',
 "['Wellness', 'Lage', 'Sauberkeit']")